# CosMx human lung

This notebook is used to tokenize the CosMx human lung dataset and generate dataset statistics for figures in nicheformer. 

## Imports and definitions

In [3]:
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import math
import numba
from scipy.sparse import issparse
from sklearn.utils import sparsefuncs

import pyarrow.parquet as pq
import pyarrow
from os.path import join
from tqdm import tqdm

In [4]:
modality_dict = {
    'dissociated': 3,
    'spatial': 4,}

specie_dict = {
    'human': 5,
    'Homo sapiens': 5,
    'Mus musculus': 6,
    'mouse': 6,}

technology_dict = {
    "merfish": 7,
    "MERFISH": 7,
    "cosmx": 8,
    "NanoString digital spatial profiling": 8,
    "visium": 9,
    "10x 5' v2": 10,
    "10x 3' v3": 11,
    "10x 3' v2": 12,
    "10x 5' v1": 13,
    "10x 3' v1": 14,
    "10x 3' transcription profiling": 15, 
    "10x transcription profiling": 15,
    "10x 5' transcription profiling": 16,
    "CITE-seq": 17, 
    "Smart-seq v4": 18,
}

author_cell_type_dict = {
    'B-cell': 0, 
    'NK': 1, 
    'T CD4 memory': 2, 
    'T CD4 naive': 3, 
    'T CD8 memory': 4,
    'T CD8 naive': 5, 
    'Treg': 6, 
    'endothelial': 7, 
    'epithelial': 8, 
    'fibroblast': 9,
    'mDC': 10, 
    'macrophage': 11, 
    'mast': 12, 
    'monocyte': 13, 
    'neutrophil': 14, 
    'pDC': 15,
    'plasmablast': 16, 
    'tumor 12': 17, 
    'tumor 13': 18, 
    'tumor 5': 19, 
    'tumor 6': 20,
    'tumor 9': 21
}

niche_label_dict = {
    'immune': 0, 
    'lymphoid structure': 1, 
    'macrophages': 2,
    'myeloid-enriched stroma': 3, 
    'neutrophils': 4,
    'plasmablast-enriched stroma': 5, 
    'stroma': 6, 
    'tumor interior': 7,
    'tumor-stroma boundary': 8
}

region_label_dict = {
    'nan': 0
}

## Paths

In [2]:
BASE_PATH = '../../data/model_means'
DATA_PATH = '' # specify path to raw nicheformer-data object for the cosmy human lung data
OUT_PATH = '' # specify saving  path

## Tokenization functions

In [5]:
def sf_normalize(X):
    X = X.copy()
    counts = np.array(X.sum(axis=1))
    # avoid zero devision error
    counts += counts == 0.
    # normalize to 10000. counts
    scaling_factor = 10000. / counts

    if issparse(X):
        sparsefuncs.inplace_row_scale(X, scaling_factor)
    else:
        np.multiply(X, scaling_factor.reshape((-1, 1)), out=X)

    return X

@numba.jit(nopython=True, nogil=True)
def _sub_tokenize_data(x: np.array, max_seq_len: int = -1, aux_tokens: int = 30):
    scores_final = np.empty((x.shape[0], max_seq_len if max_seq_len > 0 else x.shape[1]))
    for i, cell in enumerate(x):
        nonzero_mask = np.nonzero(cell)[0]    
        sorted_indices = nonzero_mask[np.argsort(-cell[nonzero_mask])][:max_seq_len] 
        sorted_indices = sorted_indices + aux_tokens # we reserve some tokens for padding etc (just in case)
        if max_seq_len:
            scores = np.zeros(max_seq_len, dtype=np.int32)
        else:
            scores = np.zeros_like(cell, dtype=np.int32)
        scores[:len(sorted_indices)] = sorted_indices.astype(np.int32)
        
        scores_final[i, :] = scores
        
    return scores_final


def tokenize_data(x: np.array, median_counts_per_gene: np.array, max_seq_len: int = None):
    """Tokenize the input gene vector to a vector of 32-bit integers."""

    x = np.nan_to_num(x) # is NaN values, fill with 0s
    x = sf_normalize(x)
    median_counts_per_gene += median_counts_per_gene == 0
    out = x / median_counts_per_gene.reshape((1, -1))

    scores_final = _sub_tokenize_data(out, 4096, 30)

    return scores_final.astype('i4')

## Loading model with right gene ordering

In [6]:
model = sc.read_h5ad("nicheformer/data/model_means/model.h5ad")

## Technology mean

In [7]:
cosmx_mean = np.load("/home/ec2-user/SageMaker/nicheformer/data/model_means/cosmx_mean_script.npy")

In [8]:
cosmx_mean = np.nan_to_num(cosmx_mean)
rounded_values = np.where((cosmx_mean % 1) >= 0.5, np.ceil(cosmx_mean), np.floor(cosmx_mean))
cosmx_mean = np.where(cosmx_mean == 0, 1, rounded_values)
cosmx_mean

array([1., 1., 1., ..., 1., 1., 1.])

## Loading CosMx lung data

In [3]:
cosmx = sc.read_h5ad(f"{DATA_PATH}/nanostring_lung_annotated.h5ad")
cosmx

AnnData object with n_obs × n_vars = 771236 × 960
    obs: 'AspectRatio', 'CenterX_global_px', 'CenterY_global_px', 'Width', 'Height', 'Mean.MembraneStain', 'Max.MembraneStain', 'Mean.PanCK', 'Max.PanCK', 'Mean.CD45', 'Max.CD45', 'Mean.CD3', 'Max.CD3', 'Mean.DAPI', 'Max.DAPI', 'niche', 'image_id', 'cell_ID', 'sex_ontology_term_id', 'assay_ontology_term_id', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'suspension_type', 'tissue_type', 'condition_id', 'sample_id', 'donor_id', 'author_cell_type', 'library_key', 'region', 'assay', 'organism', 'sex', 'tissue', 'dataset', 'x', 'y', 'nicheformer_split', '_scvi_batch', '_scvi_labels'
    var: 'level_0', 'index', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'log1p', 'niche', 'nicheformer_version', 'schema_version', 'title'
    obsm: 'X_niche_0', 'X_niche_1', 'X_niche_2', 'X_niche_3', 'X_niche_4', 'X_pca', 'X_scvi', 'spatial'
    layers: 'log1p', 'raw'

## Concatenation
Next we concatenate the `model` and the `cosmx` object to ensure they are in the same order. This ensures we have the same gene ordering in the object.

In [17]:
adata = ad.concat([model, cosmx], join='outer', axis=0)
# dropping the first observation 
cosmx = adata[1:].copy()
# for memory efficiency 
del adata

In [18]:
cosmx.obs = cosmx.obs[
    ['assay', 'organism', 'nicheformer_split', 'batch', 'niche', 'region', 'author_cell_type']
]
cosmx.obs['modality'] = 'spatial'
cosmx.obs['specie'] = cosmx.obs.organism

In [19]:
cosmx.obs.replace({'specie': specie_dict}, inplace=True)
cosmx.obs.replace({'modality': modality_dict}, inplace=True)
cosmx.obs.replace({'assay': technology_dict}, inplace=True)
cosmx.obs.replace({'author_cell_type': author_cell_type_dict}, inplace=True)
cosmx.obs.replace({'niche': niche_label_dict}, inplace=True)
cosmx.obs.replace({'region': region_label_dict}, inplace=True)

## Tokenize train data

We know tokenize the train partition of the dataset. 

In [20]:
SPLIT = 'train'

In [21]:
# subsetting the anndata object
cosmx_split = cosmx[cosmx.obs.nicheformer_split == SPLIT].copy()
# dropping the index as the original index can create issues 
cosmx_split.obs.reset_index(drop=True, inplace=True)
# writing the data
cosmx_split.write(f"{OUT_PATH}/cosmx_human_lung_{SPLIT}_ready_to_tokenize.h5ad")

In [22]:
obs_cosmx_split = cosmx_split.obs
print('n_obs: ', obs_cosmx_split.shape[0])
N_BATCHES = math.ceil(obs_cosmx_split.shape[0] / 10_000)
print('N_BATCHES: ', N_BATCHES)
batch_cosmx_indices = np.array_split(obs_cosmx_split.index, N_BATCHES)
chunk_len = len(batch_cosmx_indices[0])
print('chunk_len: ', chunk_len)

n_obs:  602123
N_BATCHES:  61
chunk_len:  9871


In [23]:
obs_cosmx_split = obs_cosmx_split.reset_index().rename(columns={'index':'idx'})
obs_cosmx_split['idx'] = obs_cosmx_split['idx'].astype('i8')

In [24]:
print(f"Tokenizing {SPLIT}")
for batch in tqdm(range(N_BATCHES)):
    obs_tokens = obs_cosmx_split.iloc[batch*chunk_len:chunk_len*(batch+1)].copy()
    cosmx_tokenized = tokenize_data(cosmx_split.X[batch*chunk_len:chunk_len*(batch+1)], cosmx_mean, 4096)

    obs_tokens = obs_tokens[['assay', 'specie', 'modality', 'idx', 'author_cell_type', 'niche', 'region']]
    # concatenate dataframes
    
    obs_tokens['X'] = [cosmx_tokenized[i, :] for i in range(cosmx_tokenized.shape[0])]
    
    for i in np.arange(5):
        niche = cosmx_split.obsm[f"X_niche_{i}"].toarray()[batch*chunk_len:chunk_len*(batch+1)]
        obs_tokens[f"X_niche_{i}"] = [niche[i, :] for i in range(niche.shape[0])]

    # mix spatial and dissociate data
    obs_tokens = obs_tokens.sample(frac=1)
    
    total_table = pyarrow.Table.from_pandas(obs_tokens)
    
    pq.write_table(total_table, f'{join(OUT_PATH, SPLIT)}/tokens-{batch}.parquet',
                    row_group_size=1024,)

Tokenizing train


100%|███████████████████████████████████████████| 61/61 [03:17<00:00,  3.24s/it]


In [25]:
# checking for the last object whether everything looks accurate 
obs_tokens.head(2)

,assay,specie,modality,idx,author_cell_type,niche,region,X,X_niche_0,X_niche_1,X_niche_2,X_niche_3,X_niche_4
593988,8,5,4,593988,2,8,0,"[252, 18331, 2140, 2853, 8959, 14989, 12294, 7...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 9.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 14.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0,..."
598330,8,5,4,598330,18,7,0,"[18939, 10438, 18768, 138, 16254, 12317, 1739,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."


In [26]:
pd.read_parquet(f'{join(OUT_PATH, SPLIT)}/tokens-{batch}.parquet').head(2)

,assay,specie,modality,idx,author_cell_type,niche,region,X,X_niche_0,X_niche_1,X_niche_2,X_niche_3,X_niche_4
593988,8,5,4,593988,2,8,0,"[252, 18331, 2140, 2853, 8959, 14989, 12294, 7...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 9.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 14.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0,..."
598330,8,5,4,598330,18,7,0,"[18939, 10438, 18768, 138, 16254, 12317, 1739,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."


### Reading for testing
We are reading one parquet file to test whether everything worked.

## Tokenize test data

In [27]:
SPLIT = 'test'

In [28]:
# subsetting the anndata object
cosmx_split = cosmx[cosmx.obs.nicheformer_split == SPLIT].copy()
# dropping the index as the original index can create issues 
cosmx_split.obs.reset_index(drop=True, inplace=True)
# writing the data
cosmx_split.write(f"{OUT_PATH}/cosmx_human_lung_{SPLIT}_ready_to_tokenize.h5ad")

In [29]:
obs_cosmx_split = cosmx_split.obs
print('n_obs: ', obs_cosmx_split.shape[0])
N_BATCHES = math.ceil(obs_cosmx_split.shape[0] / 10_000)
print('N_BATCHES: ', N_BATCHES)
batch_cosmx_indices = np.array_split(obs_cosmx_split.index, N_BATCHES)
chunk_len = len(batch_cosmx_indices[0])
print('chunk_len: ', chunk_len)

n_obs:  169113
N_BATCHES:  17
chunk_len:  9948


In [30]:
obs_cosmx_split = obs_cosmx_split.reset_index().rename(columns={'index':'idx'})
obs_cosmx_split['idx'] = obs_cosmx_split['idx'].astype('i8')

In [31]:
print(f"Tokenizing {SPLIT}")
for batch in tqdm(range(N_BATCHES)):
    obs_tokens = obs_cosmx_split.iloc[batch*chunk_len:chunk_len*(batch+1)].copy()
    cosmx_tokenized = tokenize_data(cosmx_split.X[batch*chunk_len:chunk_len*(batch+1)], cosmx_mean, 4096)

    obs_tokens = obs_tokens[['assay', 'specie', 'modality', 'idx', 'author_cell_type', 'niche', 'region']]
    # concatenate dataframes
    
    obs_tokens['X'] = [cosmx_tokenized[i, :] for i in range(cosmx_tokenized.shape[0])]
    
    for i in np.arange(5):
        niche = cosmx_split.obsm[f"X_niche_{i}"].toarray()[batch*chunk_len:chunk_len*(batch+1)]
        obs_tokens[f"X_niche_{i}"] = [niche[i, :] for i in range(niche.shape[0])]

    # mix spatial and dissociate data
    obs_tokens = obs_tokens.sample(frac=1)
    
    total_table = pyarrow.Table.from_pandas(obs_tokens)
    
    pq.write_table(total_table, f'{join(OUT_PATH, SPLIT)}/tokens-{batch}.parquet',
                    row_group_size=1024,)

Tokenizing test


100%|███████████████████████████████████████████| 17/17 [00:44<00:00,  2.60s/it]


In [32]:
obs_tokens.head(2)

,assay,specie,modality,idx,author_cell_type,niche,region,X,X_niche_0,X_niche_1,X_niche_2,X_niche_3,X_niche_4
167916,8,5,4,167916,14,6,0,"[10185, 12120, 5542, 8995, 1263, 7546, 5603, 2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 2.0, 0.0, ..."
162269,8,5,4,162269,9,4,0,"[13247, 9783, 15535, 3588, 4693, 12391, 7546, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 1.0, ..."


In [33]:
pd.read_parquet(f'{join(OUT_PATH, SPLIT)}/tokens-0.parquet').head(2)

,assay,specie,modality,idx,author_cell_type,niche,region,X,X_niche_0,X_niche_1,X_niche_2,X_niche_3,X_niche_4
4474,8,5,4,4474,3,1,0,"[15953, 4197, 12403, 12945, 3401, 6295, 3879, ...","[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[3.0, 0.0, 0.0, 6.0, 0.0, 2.0, 0.0, 1.0, 0.0, ...","[9.0, 0.0, 0.0, 8.0, 0.0, 3.0, 0.0, 1.0, 0.0, ...","[19.0, 1.0, 0.0, 22.0, 0.0, 5.0, 1.0, 1.0, 0.0...","[27.0, 2.0, 0.0, 26.0, 0.0, 7.0, 3.0, 4.0, 0.0..."
8386,8,5,4,8386,8,3,0,"[12152, 15731, 344, 6071, 7428, 12764, 16848, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 3.0, ...","[0.0, 0.0, 1.0, 4.0, 0.0, 0.0, 1.0, 2.0, 4.0, ...","[0.0, 2.0, 1.0, 7.0, 0.0, 2.0, 1.0, 7.0, 5.0, ...","[2.0, 4.0, 1.0, 13.0, 0.0, 3.0, 1.0, 16.0, 8.0..."


In [20]:
config['data_path']

'/home/ec2-user/SageMaker/test_data/spatial/preprocessed/Xenium_Preview_Human_Non_diseased_Lung_With_Add_on_FFPE_outs.h5ad'

In [16]:
adata = ad.read_h5ad(config['data_path'])
technology_mean = np.load(config['technology_mean_path'])
 
# format data properly with the model
#adata = ad.concat([model, adata], join='outer', axis=0)
# dropping the first observation 
adata = adata[1:].copy()

In [22]:
adata.var.index

Index(['ENSG00000159640', 'ENSG00000130234', 'ENSG00000213088',
       'ENSG00000115091', 'ENSG00000151694', 'ENSG00000042980',
       'ENSG00000154734', 'ENSG00000123146', 'ENSG00000162618',
       'ENSG00000204305',
       ...
       'ENSG00000126756', 'ENSG00000167397', 'ENSG00000160948',
       'ENSG00000155659', 'ENSG00000110799', 'ENSG00000105583',
       'ENSG00000109501', 'ENSG00000105989', 'ENSG00000109046',
       'ENSG00000184937'],
      dtype='object', length=392)

In [23]:
np.size(technology_mean)

20310

In [24]:
adata1 = ad.concat([model, adata], join='outer', axis=0)

In [25]:
adata1.var.index

Index(['ENSG00000000003', 'ENSG00000000005', 'ENSG00000000419',
       'ENSG00000000457', 'ENSG00000000460', 'ENSG00000000938',
       'ENSG00000000971', 'ENSG00000001036', 'ENSG00000001084',
       'ENSG00000001167',
       ...
       'ENSMUSG00000112117', 'ENSMUSG00000112148', 'ENSMUSG00000112276',
       'ENSMUSG00000113186', 'ENSMUSG00000114028', 'ENSMUSG00000114469',
       'ENSMUSG00000115424', 'ENSMUSG00000115432', 'ENSMUSG00000115529',
       'ENSMUSG00000115965'],
      dtype='object', length=20311)

In [32]:
names=adata.var.index.tolist()

In [28]:
technology_mean

array([        nan,         nan,         nan, ..., 14.51111087,
               nan, 14.23553126])

In [29]:
model_ad = ad.read_h5ad('nicheformer/data/model_means/model.h5ad')

In [34]:
full_names= model_ad.var.index.tolist()

In [35]:
unique_values = [x for x in names if x not in full_names]

In [36]:
unique_values

['ENSG00000229415']